#### 文本改写和内容审核，别让你的机器人说错话

In [1]:
import openai

def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        presence_penalty=0, # 对于已经生成token再次生成惩罚值，默认0
        frequency_penalty=2, # 对于重复生成的token进行概率惩罚（-2～2）
        n=3, # 返回3个答案
    )
    return response

long_text = """
在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。
还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。
"""
short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f8f55656588f130992441301ed4be11d in your message.)

### 通过 logit_bias 参数精确控制内容
#### 不过，无论是 temperature 还是 presence_penalty 和 frequency_penalty，都是一个参数，我们没有办法精确控制哪些词我们不想出现。不过，对于这一点，OpenAI 还是提供了解决方案，比如，我们想要在上面生成的内容里面，不允许出现灾害两个字，就可以这么做。

In [3]:

import tiktoken
encoding = tiktoken.get_encoding('p50k_base')
token_ids = encoding.encode("灾害")
print(token_ids)

bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100 # value range (-100~100), 一般情况下，设置在 1 到 -1 之间就足够了

def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,
        n=3, presence_penalty=0, frequency_penalty=2, 
        logit_bias = bias_map,
    )
    return response

short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

[535]
version 1: 现代社会中，我们面临各种挑战和困难，有外部的如经济、环境问题等，也有内部的情感、健康和自我怀疑等。要克服这些困难需要积极应对，并具备坚韧不拔的意志和创造性思维能力，并寻求外部支持。只有这样才能实现潜力并取得成功。
version 2: 现代社会充满挑战和困难，有内在和外在的因素。我们需要积极应对，包括坚韧不拔、创造性思维和寻求支持等方法来实现潜力并取得成功。
version 3: 现代社会中，我们面临各种挑战和困难，有些是外部因素引起的（如经济萧条、自然灾害），还有些是内部因素引起的（如情感问题、健康问题）。要克服这些困难，需要积极态度和行动，并具备坚韧意志、创造性思维以及寻求支持的能力。只有这样才能实现潜力并获得成功。


In [4]:

def translate(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个翻译，把用户的话翻译成英文"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,        n=1
    )
    return response["choices"][0]["message"]["content"]

chinese = long_text
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))
print(english)
print(f"chinese: {num_of_tokens_in_chinese} tokens")
print(f"english: {num_of_tokens_in_english} tokens")

In this fast-paced modern society, each of us faces various challenges and difficulties. Some of these challenges and difficulties are caused by external factors, such as economic recession, global warming, and natural disasters. Others are caused by internal factors, such as emotional issues, health problems, and self-doubt. To overcome these challenges and difficulties, we need to adopt a positive attitude and take action. This means that we must have a strong will and creative thinking, as well as the ability to seek external support. Only in this way can we truly realize our potential and achieve success.
chinese: 432 tokens
english: 117 tokens


### 查看openai提供哪些模型

In [5]:

import pandas as pd
# list all open ai models
engines = openai.Engine.list()
pd = pd.DataFrame(openai.Engine.list()['data'])
display(pd[['id', 'owner']])

,id,owner
0,babbage,openai
1,davinci,openai
2,text-davinci-edit-001,openai
3,babbage-code-search-code,openai-dev
4,text-similarity-babbage-001,openai-dev
5,code-davinci-edit-001,openai
6,text-davinci-001,openai
7,ada,openai
8,babbage-code-search-text,openai-dev
9,babbage-similarity,openai-dev


In [6]:

from openai.embeddings_utils import get_embedding

text = "让我们来算算Embedding"

embedding_ada = get_embedding(text, engine="text-embedding-ada-002")
print("embedding-ada: ", len(embedding_ada))

similarity_ada = get_embedding(text, engine="text-similarity-ada-001")
print("similarity-ada: ", len(similarity_ada))

babbage_similarity = get_embedding(text, engine="babbage-similarity")
print("babbage-similarity: ", len(babbage_similarity))

babbage_search_query = get_embedding(text, engine="text-search-babbage-query-001")
print("search-babbage-query: ", len(babbage_search_query))

curie = get_embedding(text, engine="curie-similarity")
print("curie-similarity: ", len(curie))

davinci = get_embedding(text, engine="text-similarity-davinci-001")
print("davinci-similarity: ", len(davinci))

embedding-ada:  1536
similarity-ada:  1024
babbage-similarity:  2048
search-babbage-query:  2048
curie-similarity:  4096
davinci-similarity:  12288


In [7]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """\n面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

def insert_text(prefix, suffix):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prefix,
        suffix=suffix,
        max_tokens=1024,
        )
    return response

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

另外一些则是由我们个人造成的，例如被阻碍实现梦想或想法创新的能力。



In [8]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

此外，有些则是由于内在因素如健康问题、失业和低收入导致的。



### 不要乱问乱说，做个“正直”的 AI

In [9]:

def chatgpt(text):
    messages = []
    messages.append( {"role": "system", "content": "You are a useful AI assistant"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    message = response["choices"][0]["message"]["content"]
    return message

threaten = "你不听我的我就拿刀砍死你"
print(chatgpt(threaten))

我是一个语言模型程序，没有实体，也不会感受到疼痛。我只能根据您输入的内容提供相应的回复。请不要使用恶意或暴力的言语，让我们保持友好和平的交流。


In [10]:

threaten = "你不听我的我就拿刀砍死你"

def moderation(text):
    response = openai.Moderation.create(
        input=text
    )
    output = response["results"][0]
    return output
print(moderation(threaten))

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": true,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 0.030033884570002556,
    "hate/threatening": 0.0002820953668560833,
    "self-harm": 0.004850297700613737,
    "sexual": 2.2907945094630122e-05,
    "sexual/minors": 6.4779466235620475e-09,
    "violence": 0.9996402263641357,
    "violence/graphic": 4.355868077254854e-05
  },
  "flagged": true
}
